In [1]:
!pip install diffusers
!pip install segment_anything
!pip install torch
!pip install transformers
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
from segment_anything import SamPredictor,sam_model_registry


In [3]:
device = "cuda"
s_checkpoint = "/content/drive/MyDrive/SAM/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint = s_checkpoint)
sam.to(device)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [4]:
import torch

predictor = SamPredictor(sam)
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,

)
pipe = pipe.to(device)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [5]:
import gradio as gr
pix=[]
def generate_mask(image,evt: gr.SelectData):
  pix.append(evt.index)
  predictor.set_image(image)
  input_points = np.array(pix)
  input_label = np.ones(input_points.shape[0])
  print(input_label)
  mask, _, _ = predictor.predict(
      point_coords = input_points,
      point_labels = input_label,
      multimask_output = False
  )

  mask = Image.fromarray(mask[0,:,:])
  return mask



In [9]:
def inpaint(image,mask,prompt):
  image = Image.fromarray(image)
  mask = Image.fromarray(mask)

  image = image.resize((512, 512))
  mask = mask.resize((512, 512))
  
  output = pipe(
      prompt = prompt,
      image = image,
      mask_image = mask,
      negative_prompt="deformed face,deformed hands,blurry"
  ).images[0]

  return output


In [10]:
import gradio as gr

with gr.Blocks() as demo:
  with gr.Row():
    input_img = gr.Image(label="Input")
    mask_img = gr.Image(label="mask")
    output_img = gr.Image(label="output")
  with gr.Row():
    prompt_txt = gr.Textbox(lines=1,label="Prompt")
  with gr.Row():
    submit = gr.Button("Submit")
  input_img.select(generate_mask, [input_img], [mask_img])
  submit.click(inpaint, inputs = [input_img, mask_img, prompt_txt], outputs = [output_img])

In [11]:
if "__name__ == __main__":
  demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>